# Azure Vision Implementaion - Dima 

This notebook utilizes Azure AI Document Intelligence Studio to extract text from a set of Herbarium specimens, obtained from: https://www.gbif.org/occurrence/gallery?basis_of_record=PRESERVED_SPECIMEN&media_ty%5B%E2%80%A6%5Daxon_key=6&year=1000,1941&advanced=1&occurrence_status=present

A selection of 30 specimens was downloaded to the /projectnb/sparkgrp/ml-herbarium-grp/fall2023/LLM_Specimens folder. 

The folder is made up of:
1) 20 images that contain pure text, ranging from plain to hard-to-read-cursive and 1
2) 10 images that contain both the visual plant specimen and the attached textual labels

Special care was taken to select a diverse collection of specimens, ranging in text quality and type

In regards to the 10 images: there was a general trend in that the images with plant specimens and the actual text, the text was too small and or blurry to be deciphered by any LLM. Next steps would include improving the quality of the text for the LLM to analyze it. 

Currently: the notebook takes an input image from: /projectnb/sparkgrp/ml-herbarium-grp/fall2023/LLM_Specimens, runs it through Azure Vision, analyzes all text, creates a pdf with the original image, an annotated image that has boxes around identified words and predicted words written over the original text. Below the image the entire text identified is printed along with the confidence score for each identified term. All this is saved and stored in: /projectnb/sparkgrp/ml-herbarium-grp/fall2023/AzureVision-results

Immediate next steps:

1. Obtain a student Microsoft Azure account to finish the work (testing was done with a personal account, ran out of free credits)
2. Improve annotated images- currently the predicted text is hard to read, going to change it so that its above the orginal words. 
3. Integrate GPT-4 to parse the written text into a format that clearly returns the species, date collected, geography. 


In [16]:
#!pip install azure-ai-formrecognizer --pre
#!pip install opencv-python-headless matplotlib
#!pip install matplotlib pillow
#!pip install ipywidgets
#!pip install shapely
#!pip install openai
#!pip install reportlab

In [15]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw, ImageFont
import openai
import re
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas


# Azure Cognitive Services endpoint and key
endpoint = "https://herbariumsamplerecognition.cognitiveservices.azure.com/"
key = "d341921d724e44bda113bc343e88d476"

def sanitize_filename(filename):
    # Remove characters that are not alphanumeric, spaces, dots, or underscores
    return re.sub(r'[^\w\s\.-]', '', filename)

def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in bounding_box])

def draw_boxes(image_path, words):
    original_image = Image.open(image_path)
    annotated_image = original_image.copy()
    draw = ImageDraw.Draw(annotated_image)

    for word in words:
        polygon = word['polygon']
        if polygon:
            bbox = [(point.x, point.y) for point in polygon]
            try:
                # Replace special characters that cannot be encoded in 'latin-1'
                text_content = word['content'].encode('ascii', 'ignore').decode('ascii')
            except Exception as e:
                print(f"Error processing text {word['content']}: {e}")
                text_content = "Error"
            draw.polygon(bbox, outline="red")
            draw.text((bbox[0][0], bbox[0][1]), text_content, fill="green")
    
    return annotated_image


def parse_document_content(content):
    openai.api_key = 'your-api-key'

    try:
        response = openai.Completion.create(
            model="gpt-4",
            prompt=f"Extract specific information from the following text: {content}\n\nSpecies Name: ",
            max_tokens=100
            # Add additional parameters as needed
        )
        parsed_data = response.choices[0].text.strip()
        return parsed_data
    except Exception as e:
        print("An error occurred:", e)
        return None


def analyze_read(image_path, output_path, show_first_output=False):
    try:
        with open(image_path, "rb") as f:
            image_stream = f.read()

        document_analysis_client = DocumentAnalysisClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)
        )

        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-read", image_stream)
        result = poller.result()

       # Collect words, their polygon data, and confidence
        words = []
        confidence_text = ""
        for page in result.pages:
            for word in page.words:
                words.append({
                    'content': word.content,
                    'polygon': word.polygon
                })
                confidence_text += "'{}' confidence {}\n".format(word.content, word.confidence)

        document_content = result.content + "\n\nConfidence Metrics:\n" + confidence_text
        #parsed_info = parse_document_content(document_content)

        original_image = Image.open(image_path)
        annotated_img = draw_boxes(image_path, words)

        # Set up PDF
        output_filename = os.path.join(output_path, sanitize_filename(os.path.basename(image_path).replace('.png', '.pdf')))
        c = canvas.Canvas(output_filename, pagesize=letter)
        width, height = letter  # usually 612 x 792

        # Draw original image
        if original_image.height <= height:
            c.drawImage(image_path, 0, height - original_image.height, width=original_image.width, height=original_image.height, mask='auto')
            y_position = height - original_image.height
        else:
            # Handle large images or add scaling logic here
            pass

        # Draw annotated image
        annotated_image_path = '/tmp/annotated_image.png'  # Temporary path for the annotated image
        annotated_img.save(annotated_image_path)
        if y_position - annotated_img.height >= 0:
            c.drawImage(annotated_image_path, 0, y_position - annotated_img.height, width=annotated_img.width, height=annotated_img.height, mask='auto')
            y_position -= annotated_img.height
        else:
            c.showPage()  # Start a new page if not enough space
            c.drawImage(annotated_image_path, 0, height - annotated_img.height, width=annotated_img.width, height=annotated_img.height, mask='auto')
            y_position = height - annotated_img.height

        # Add text
        textobject = c.beginText()
        textobject.setTextOrigin(10, y_position - 15)
        textobject.setFont("Times-Roman", 12)

        for line in document_content.split('\n'):
            if textobject.getY() - 15 < 0:  # Check if new page is needed for more text
                c.drawText(textobject)
                c.showPage()
                textobject = c.beginText()
                textobject.setTextOrigin(10, height - 15)
                textobject.setFont("Times-Roman", 12)
            textobject.textLine(line)
        
        c.drawText(textobject)
        c.save()

        # Show the first output
        if show_first_output:
            os.system(f"open {output_filename}")

    except Exception as e:
        print(f"An error occurred while processing {image_path}: {e}")


if __name__ == "__main__":
    input_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/LLM_Specimens'
    output_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/AzureVision-results'
    first_output_shown = False

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over each image in the input folder
    for image_file in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_file)
        
        # Check if the file is an image
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            analyze_read(image_path, output_folder, show_first_output=not first_output_shown)
            first_output_shown = True  # Ensure that only the first output is shown


Couldn't get a file descriptor referring to the console


An error occurred while processing /projectnb/sparkgrp/ml-herbarium-grp/fall2023/LLM_Specimens/Text_Sample_19.png: (403) Out of call volume quota for FormRecognizer F0 pricing tier. Please retry after 6 days. To increase your call volume switch to a paid tier.
Code: 403
Message: Out of call volume quota for FormRecognizer F0 pricing tier. Please retry after 6 days. To increase your call volume switch to a paid tier.
An error occurred while processing /projectnb/sparkgrp/ml-herbarium-grp/fall2023/LLM_Specimens/Text_Sample_10.png: (403) Out of call volume quota for FormRecognizer F0 pricing tier. Please retry after 6 days. To increase your call volume switch to a paid tier.
Code: 403
Message: Out of call volume quota for FormRecognizer F0 pricing tier. Please retry after 6 days. To increase your call volume switch to a paid tier.
An error occurred while processing /projectnb/sparkgrp/ml-herbarium-grp/fall2023/LLM_Specimens/Text_Sample_6.png: (403) Out of call volume quota for FormRecogni